In [7]:
import pandas as pd
import spacy
from pathlib import Path

In [8]:
MAIN_PATH = Path(r"C:\Users\lhi30\Haein\2023\YBIGTA\2023-2\DA\Wiki_People\Share")

In [75]:
test_df

,Unnamed: 0,0,1,2,3
0,0,1911,.mw-parser-output .sidebar{width:22em;float:r...,summary,NaN
1,1,1922,"Apart from two years between 1922 and 1924, he...",summary,NaN
2,2,1940,Ideologically an adherent to economic liberali...,summary,NaN
3,3,1904,He was a member of the Liberal Party from 1904...,summary,NaN
4,4,1895,He joined the British Army in 1895 and saw act...,summary,NaN
...,...,...,...,...,...
2153,2153,2010,"Seldon, Anthony (2010).",Further reading,NaN
2154,2154,1951,Churchill's Indian Summer: The Conservative Go...,Further reading,NaN
2155,2155,2021,"Smith, Gary Scott (2021).",Further reading,NaN
2156,2156,8028,ISBN 978-0-8028-7700-0.,Further reading,NaN


In [87]:
test_df = pd.read_csv(MAIN_PATH / "event_extraction/Winston_Churchill.csv")
test_df = test_df[test_df.iloc[:, 3] == "summary"]

In [112]:
test = test_df.iloc[18, 2]
test

'Ideologically an adherent to economic liberalism and imperialism, he was for most of his career a member of the Conservative Party, which he led from 1940 to 1955.'

In [129]:
import spacy
from spacy.matcher import Matcher
from spacy.util import filter_spans

nlp = spacy.load("en_core_web_trf")


In [131]:
doc = nlp(test)
# for token in doc:
#     if token.pos_ in ["VERB"]:
#         print(token.text)
# for chunk in doc.noun_chunks:
#     print(chunk.text,)
# for ent in doc.ents:
#     print(ent.text, ent.label_)

# print("\n")
for token in doc:
        print(token.text, token.pos_, token.dep_)

Ideologically ADV advmod
an DET det
adherent NOUN advcl
to ADP prep
economic ADJ amod
liberalism NOUN pobj
and CCONJ cc
imperialism NOUN conj
, PUNCT punct
he PRON nsubj
was AUX ROOT
for ADP prep
most ADJ pobj
of ADP prep
his PRON poss
career NOUN pobj
a DET det
member NOUN attr
of ADP prep
the DET det
Conservative PROPN compound
Party PROPN pobj
, PUNCT punct
which PRON dobj
he PRON nsubj
led VERB relcl
from ADP prep
1940 NUM pobj
to ADP prep
1955 NUM pobj
. PUNCT punct


In [160]:
# Define the pattern
pattern = [
    {"DEP": {"in": ["nsubj", "nsubjpass"]}, "OP": "+"},
    {"POS": {"in": ["VERB", "AUX"]}, "OP": "+"},
    {"OP": "{0,10}"},
    {"DEP": {"in": ["dobj", "pobj"]}, "OP": "*"},
]

# Instantiate a Matcher instance
matcher = Matcher(nlp.vocab)
matcher.add("CustomPattern", [pattern])


# Apply the matcher to the processed document
matches = matcher(doc)

result_list = []
for match_id, start, end in matches:
    # Get the tokens that match the pattern
    match_tokens = doc[start:end]
    
    # Extract relevant information
    subj_verb_obj = [token.text for token in match_tokens if token.dep_ in ["nsubj", "nsubjpass", "VERB", "AUX", "dobj", "pobj"]]
    
    # Create a list with the extracted information
    result_list.append(subj_verb_obj)

# Print the result
print(result_list)

[['he'], ['he'], ['he', 'most'], ['he', 'most'], ['he', 'most'], ['he', 'most', 'career'], ['he', 'most', 'career'], ['he', 'most', 'career'], ['he', 'most', 'career'], ['he', 'most', 'career'], ['he', 'most', 'career'], ['he', 'most', 'career', 'Party'], ['he'], ['he'], ['he', '1940'], ['he', '1940'], ['he', '1940', '1955'], ['he', '1940', '1955']]


In [161]:
matches

[(281905389811269810, 9, 11),
 (281905389811269810, 9, 12),
 (281905389811269810, 9, 13),
 (281905389811269810, 9, 14),
 (281905389811269810, 9, 15),
 (281905389811269810, 9, 16),
 (281905389811269810, 9, 17),
 (281905389811269810, 9, 18),
 (281905389811269810, 9, 19),
 (281905389811269810, 9, 20),
 (281905389811269810, 9, 21),
 (281905389811269810, 9, 22),
 (281905389811269810, 24, 26),
 (281905389811269810, 24, 27),
 (281905389811269810, 24, 28),
 (281905389811269810, 24, 29),
 (281905389811269810, 24, 30),
 (281905389811269810, 24, 31)]

['led']

In [107]:
from spacy import displacy
displacy.render(doc, style='dep')